# chain 链

链是LangChain的核心。它们是一系列组件，按特定顺序执行。

## 一个最简单的例子: prompt + model + output parser
目前官网推荐使用 LCEL 方式来构建 Chain，即 LangChain Expression Language。

需要配置 openai key

In [28]:
%env OPENAI_API_KEY=you-api-key

env: OPENAI_API_KEY=you-api-key


如果无法访问 openai api，可以配置 base url

In [29]:
%env OPENAI_API_BASE=you-base-url

env: OPENAI_API_BASE=you-base-url


我们可以额外定义一个函数去计算每次使用的 tokens 数量，这样我们就可以方便地查看每次调用 api 所消耗的 tokens

In [15]:
def chain_invoke_tokens(chain, prompt):
    with get_openai_callback() as cb:
        result = chain.invoke(prompt)
        print(f'Spent a total of {cb.total_tokens} tokens\n')

    return result

使用聊天模板，gpt3.5 turbo 模型，构建一个简单的 llm 链。

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks import get_openai_callback

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(
    chain_invoke_tokens(chain, {"topic": "ice cream"})
)

Spent a total of 39 tokens
Why did the ice cream go to therapy? 

Because it had too many toppings and couldn't get its scoop together!


为了深入了解LangChain应用程序的内部工作，可以使用 set_verbose 和 set_debug 来了解引擎执行的细节。

In [12]:
from langchain.globals import set_verbose, set_debug

set_verbose(True)
set_debug(True)

加入 PythonREPL 工具，用于数学计算，构建新的链。使用 Google 的 Gemini 模型来演示

配置 api_key

In [1]:
%env GOOGLE_API_KEY=you-api-key

env: GOOGLE_API_KEY=you-api-key


如果无法访问 api，可以配置 endpoint

In [2]:
%env GOOGLE_API_ENDPOINT=you-api-endpoint

env: GOOGLE_API_ENDPOINT=you-api-endpoint


引入需要的组件库

In [4]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL

配置 prompt 模版

In [5]:
template = """Write some python code to solve the user's problem. 

Return only python code in Markdown format, e.g.:

```python
....
```"""

prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

配置 llm 模型

In [10]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0, google_api_key=os.environ['GOOGLE_API_KEY'], transport="rest", client_options={"api_endpoint": os.environ['GOOGLE_API_ENDPOINT']}, convert_system_message_to_human=True)

定义一个格式化的函数，用于提取输出的文本

In [7]:
def _sanitize_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

构建 chain 链，并执行

In [13]:
chain = prompt | llm | StrOutputParser() | _sanitize_output | PythonREPL().run

chain.invoke({"input": "whats 2 plus 2"})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": "whats 2 plus 2"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": "whats 2 plus 2"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System: Write some python code to solve the user's problem. \n\nReturn only python code in Markdown format, e.g.:\n\n```python\n....\n```\nHuman: whats 2 plus 2"
  ]
}


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 POST https://ai.goi.oowan.net/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised InternalServerError: 500 POST https://ai.goi.oowan.net/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised GatewayTimeout: 504 POST https://ai.goi.oowan.net/v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: An error occurred with your

[llm/end] [1:chain:RunnableSequence > 3:llm:ChatGoogleGenerativeAI] [41.91s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "```python\nprint(2 + 2)\n```",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE",
              "blocked": false
            }
          ]
        },
        "type": "ChatGeneration",
        "message":

'4\n'